# Projektrapport AI-23 Deep Learning
## Finetuning av existerande LLM och modelljämförelse av sentiment analys
### Namn:
### Projektbeskrivning:
- Använda en existerande Large Language Modell (LLM), avsedd för sentiment analys, och finetunea den för IMDB reviews.
- Binära klasser, dvs "positvt" eller "negativt". (Vi diskuterade också att utöka det med "multi class" men vi hann inte göra det.)
- Jämföra olika modellers prestanda map accuracy.
### Genomförande:
- Vi använde Huggingface för att hitta användbara modeller. Det finns oerhört många olika modeller att välja mellan.
- Vi valde att jobba med dessa transformer modeller från Huggingface:
1. ***sarahai/movie-sentiment-analysis*** (modell specifikt avsedd för sentimentanalys av IMDB dataset)
2. ***distilbert-base-uncased-finetuned-sst-2-english*** ("distilbert") Den här modellen är en checkpoint på DistilBERT-base-uncased, finetunead på SST-2 (Stanford Sentiment Treebank). DistilBERT är en engelsk språklig modell förtränad på samma data som BERT (Toronto Book Corpus och hela engelska Wikipedia) med hjälp av distillation under övervakning av bert-base-uncased. Modellen har 6 lager, 768 dimensioner, 12 huvuden och totalt 66 miljoner parametrar. 
3. ***michelecafagna26/gpt2-medium-finetuned-sst2-sentiment*** (GPT-2 modell tunead för sentimentanalys)
  
- Vi valde att även använda OpenAI API, dels för direkt sentimentanalys, men även för att summera texter som var för långa för att fungera med transformer modellerna utan truncation.
4. GPT-3.5 (för direkt sentimentanalys och summering för att passa till transformer modellerna)
5. GPT-4o mini (för direkt sentimentanalys)
6. GPT-4 (för direkt sentimentanalys, men på grund av begränsningar i antal tokens per minut osv, så blev inte denna analysen klar)
#### Datahantering:
##### Format på y-feature
- Formatet på y-featuren var vi tvugna att tillse att den var på lämpligt format, tex '1' eller '0' osv. 
##### Truncation:
- (Nästan) alla modeller har begränsad antal tokens som input. Vanligast bland de modeller som vi hittat är 512 tokens. Det innebär att längre texter inte "får plats". I dessa fall kan man välja att trunkera input'en. Det innebär att slutet på texten klipps bort och ej blir en del av analysen. Det var det vanligaste metoden vi använde under projektet. 
##### Summering:
- En aalternativ approach till trunkering är att summera dem. Vi provade en modell från Huggingface (Falconsai/text_summarization). Den upplevdes som långsam och dålig.  
- Som alternativ till en lokal modell för summering, så skickade vi texter som är längre än 512 tokens till OpenAI API (GPT-3.5). Man kan specificera vilken max längd på tokens man vill ha tillbaka, vilket är bra för vårt syfte.
- Att använda GPT-3.5 för summering fungera riktigt bra. Den är relativt snabb och den kostar inte mycket. Att begära hundratals summeringar (eller sentiment-analys) kostar cirka 0.1 dollar. Att använda GPT-4 kostar minst 1 ggr mer. Ett alternativ är GPT-4 mini, som också är relativt billig.
- Vi haden en hypotes att det skulle ge en bättre sentimentanalys, jämfört med trunkering, men det syns inte i våra resultat.
#### Finetuning:
- Finetuning innebär att justera (vikterna / bias i) en redan tränad modell. Vi valde att använda distilbert som grundmodell, och finetunea den på IMDB datasetet. Initialt provade vi att använda TensorFlow, men då fungerade inte GPU-accelereringen. Därför bytte vi till PyTorch.
- Vi använde en dator med GPU (2GB minne) för att göra finetuningen av modellen. Trots det, så upplevde vi begränsningar i hur mycket vi kunde lasta datorn med. Vi var tvugna att begränsa antalet epoker till 1 och antal tränings-inputs till 2000. Det innebär att det blev en väldigt begränsad finetuning av modellen.
- Den tränade modellen testades med 2000 samples, och jämfördes med den ontränade modellen. Trots den relativt lilla träningen som genomfördes, så kunde vi se en förbättring på cirka 1%-enhet.
- Vi är medvetna om att träningen inte blev utförd på hela tillgängliga datasetet (på grund av begränsningar i beräkningskapacitet), vilket gör att vi inte når full potential. Vi är medvetna om att en epok är för lite.
- Den resulterande modellens storlek var i storleksordningen > 1GB så vi kunde inte lägga den i repot.
#### Bag of words:
- Vi valde att träna en modell från scratch, för att utvärdera dess prestanda jämfört med transfomer modeller. Vi valde ett "bag of words" koncept. Bag of words innebär att meningar delas upp i dess ord (unigram) eller delar av meningen med 2 (unigram) eller fler (N) ord i följd (N-gram). Ju större N, desto mer hänsyn tas till ordföljd.
- I korthet går modellen ut på att varje "gram" (ord eller följd av ord) har en input i det neurala nätverket. Innehållet i träningsdatan används för att bygga upp översättningen från ord till numerisk data.
- Vi provade både "uni-gram" och "bi-gram" modeller.
- Map snabbhet är detta en väldigt bra modell, både att träna och för inferens, även utan GPU.
#### Gradio:
- Vi har experimenterat med ett trevligt visualiseringsverktyg som heter Gradio. Gradio är ett användarvänligt interface mot modellen, som kan deployas både lokalt och på webben.  
  

